# Smarter Prompting, Faster: Introducing Opik's Agent Optimizers

Doug Blank, Phd

* Slides are available at: [bit.ly/opik-optimizer-dsblank-slides](https://bit.ly/opik-optimizer-dsblank-slides)
* This notebook is available at: [bit.ly/opik-optimizer-dsblank](https://bit.ly/opik-optimizer-dsblank)

You will need:
1. A Google account, for running a Colab Notebook  - [google.com](https://google.com)
2. A Comet account, for seeing Opik visualizations (free!) - [comet.com](https://comet.com)
3. An OpenAI account, for using an LLM
[platform.openai.com/settings/organization/api-keys](https://platform.openai.com/settings/organization/api-keys)


## Setup

This pip-install takes about a minute.

In [1]:
%%capture
#%pip install opik-optimizer

In [2]:
import opik_optimizer
opik_optimizer.__version__

'0.8.1'

In [3]:
import opik

# Configure Opik
opik.configure()

OPIK: Opik is already configured. You can check the settings by viewing the config file at /Users/jacquesverre/.opik.config


In [4]:
import os
import getpass
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

To save time (and money) durring this demonstration, we have capture the results of a previous run of all of these LLM interactions. Our goal is to make this not cost any money. However, we of course can guarantee that. **Use at your own risk**!

To capture the perviously cached results:

In [5]:
from opik_optimizer.demo.cache import get_litellm_cache

get_litellm_cache("opik-workshop")

Inserted 0 record(s) in litellm cache


## The Dataset

In these set of experiments, we are going to use the **HotPotQA** dataset. This dataset was designed to be difficult for regular LLMs to handle. This dataset is called a "**multi-hop**" dataset because answering the questions involves multiple reasoning steps and multiple tool calls, where the LLM needs to infer relationships, combine information, or draw conclusions based on the combined context.

Example:

> "What are the capitals of the states that border California?"

You'd need to find which states border California, and then lookup each state's capital.

The dataset has about 113,000 such crowd-sourced questions that are constructed to require the introductory paragraphs of two Wikipedia articles to answer.

[1] The name "HotPot" comes from the restaurant where the authors came up with the idea of the dataset.

In [6]:
import opik_optimizer

opik_dataset = opik_optimizer.datasets.hotpot_300()

Let's take a look at some dataset items:

In [7]:
rows = opik_dataset.get_items()
rows[0]

{'id': '0197044c-d782-7735-a8cf-bffd4d19838e',
 'question': 'Are Smyrnium and Nymania both types of plant?',
 'answer': 'yes'}

In [8]:
rows[1]

{'id': '0197044c-d781-7eb0-a717-6fd6b1415476',
 'question': 'That Darn Cat! and Never a Dull Moment were both produced by what studio?',
 'answer': 'Walt Disney Productions'}

## Opik Project

All LLM traces in Opik are saved in a "project". We'll put them all in the following project name:

In [9]:
project_name = "optimize-workshop-2025"

## The Metric

Choosing a good metric for optimization is tricky. For these examples, we'll pick one that will allow us to show improvement, and also provide a gradient of scores. In general though, this metric isn't the best for optimization runs.

We'll use "Edit Distance" AKA "Levenshtein Distance":

In [10]:
from opik.evaluation.metrics import LevenshteinRatio

metric = LevenshteinRatio(project_name=project_name)

The metric takes two things: the output of the LLM and the reference (correct answer).

In [11]:
metric.score("Hello", "Hello")

OPIK: Started logging traces to the "optimize-workshop-2025" project at https://www.comet.com/opik/api/v1/session/redirect/projects/?trace_id=0197411e-4a73-7a0b-8737-e17d1cfeee6c&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==.


ScoreResult(name='levenshtein_ratio_metric', value=1.0, reason=None, metadata=None, scoring_failed=False)

In [12]:
metric.score("Hello!", "Hello")

ScoreResult(name='levenshtein_ratio_metric', value=0.9090909090909091, reason=None, metadata=None, scoring_failed=False)

The edit distance between "Hello!" and "Hello" is 1. Here is how the .91 is computed:

In [13]:
edit_distance = 1

1 - edit_distance / (len("Hello1") + len("Hello"))


0.9090909090909091

For more information see: [Levenshtein Distance](https://en.wikipedia.org/wiki/Levenshtein_distance)

## Configuation

To create the necessary configurations for using an Opik Optimizer, you'll need a metric function and a prompt.

For this example, we'll start with a pretty bad prompt... so we can optimize it!

In [14]:
from opik_optimizer import ChatPrompt

initial_prompt = ChatPrompt(
    messages=[
        {"role": "system", "content": "Provide an answer to the question"},
        {"role": "user", "content": "{answer}"}
    ]
)

We place the 'Provide an answer to the question' as the first system message in a ChatPrompt.

In [15]:
def levenshtein_ratio(dataset_item, llm_output):
    metric = LevenshteinRatio()
    return metric.score(reference=dataset_item['answer'], output=llm_output)

As you can see our metric function is a wrapper around the built-in LevenshteinRatio(). For most Opik metrics, we need two parameters.
1. 'output' - which is the output of the LLM
2. 'reference' - the correct answer (provided by the database item "answer")



## FewShotBayesianOptimizer

The FewShotBayesianOptimizer name indicates two things:

1. It will produce Chat Prompts, or FewShot examples as described in the slides.
2. Secondly, it describes how it searches for the best set of these FewShot examples.

To use this optimizer, we import it and create an instance, passing in the project name and model parameters:

In [16]:
from opik_optimizer import (
    FewShotBayesianOptimizer,
)

optimizer = FewShotBayesianOptimizer(
    project_name=project_name,
    model="openai/gpt-4o-mini",
    temperature=0.1,
    max_tokens=5000,
)

### Baseline

Before we optimize this prompt ("Provide an answer to the question") let's see what the bare prompt does by itself on the dataset:

In [17]:
score = optimizer.evaluate_prompt(
    prompt=initial_prompt,
    dataset=opik_dataset,
    metric=levenshtein_ratio,
    n_samples=100,
)
score

Evaluation:   0%|          | 0/100 [00:00<?, ?it/s]

╭─ hotpot_300 (100 samples) ─────────────╮
│                                        │
│ Total time:        00:00:09            │
│ Number of samples: 100                 │
│                                        │
│ levenshtein_ratio_metric: 0.0547 (avg) │
│                                        │
╰────────────────────────────────────────╯

Uploading results to Opik ...

View the results ]8;id=927105;https://www.comet.com/opik/api/v1/session/redirect/experiments/?experiment_id=0197411e-4d2d-735c-a078-01c186d0ef77&dataset_id=0197044c-d51d-7cff-96c5-38969b6ae708&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==\in your Opik dashboard]8;;\.

0.05473981329382088

It scored about 16% correct. [I say "percent correct" but because we are using edit distance, that isn't quite accurate. But we can think of it this way.]

Ok, let's optimize that prompt!

In [18]:
result1 = optimizer.optimize_prompt(
    prompt=initial_prompt,
    dataset=opik_dataset,
    metric=levenshtein_ratio,
    n_trials=3,
    n_samples=50
)

╭────────────────────────────────────────────────────────────────────╮
│ ● Running Opik Evaluation - Few-Shot Bayesian Optimizer            │
╰────────────────────────────────────────────────────────────────────╯


> Let's optimize the prompt:

╭─ system ───────────────────────────────────────────────────────────╮
│                                                                    │
│  Provide an answer to the question                                 │
│                                                                    │
╰────────────────────────────────────────────────────────────────────╯
╭─ user ─────────────────────────────────────────────────────────────╮
│                                                                    │
│  {answer}                                                          │
│                                                                    │
╰────────────────────────────────────────────────────────────────────╯

Using FewShotBayesianOptimizer with the para

In [19]:
result1.display()

╔═════════════════════════════════════════════ Optimization Complete ═════════════════════════════════════════════╗
║                                                                                                                 ║
║ Optimizer:         FewShotBayesianOptimizer                                                                     ║
║ Model Used:        openai/gpt-4o-mini (Temp: 0.1)                                                               ║
║ Metric Evaluated:  levenshtein_ratio                                                                            ║
║ Initial Score:     N/A                                                                                          ║
║ Final Best Score:  0.1794                                                                                       ║
║ Total Improvement: N/A (no initial score)                                                                       ║
║ Rounds Completed:  0                                                  

What did we find? The result is a series of messages:

In [20]:
result1.details["chat_messages"]

[{'role': 'system',
  'content': 'Provide an answer to the question\n\n### Few Shot Examples\nQuestion: What novel did an author who was featured in the Voices of Ghana and later became a journalist write?\nAnswer: The Gab Boys\n\nQuestion: What were the dates of the campaign that eventually led to the Treaty of Roskilde?\nAnswer: between 30 January and 8 February 1658\n\nQuestion: Are Wanding Town and Jiujiang, Guangdong both frontier towns?\nAnswer: no\n\nQuestion: The Knicks–Nuggets was the most penalized on-court fight in the NBA since an altercation that occured in what city?\nAnswer: Auburn Hills'},
 {'role': 'user', 'content': '{answer}'}]

We'll see how we can use those in a few minutes.

## MetaPromptOptimizer

The MetaPromptOptimizer uses a clever idea: have the LLM generate better prompts!

Here is the internal system meta-prompt to have the LLM generate better prompts.

```text
You are an expert prompt engineer. Your task is to improve prompts for any type of task.

Focus on making the prompt more effective by:

1. Being clear and specific about what is expected
2. Providing necessary context and constraints
3. Guiding the model to produce the desired output format
4. Removing ambiguity and unnecessary elements
5. Maintaining conciseness while being complete

Return a JSON array of prompts with the following structure:
{
    "prompts": [
        {
            "prompt": "the improved prompt text",
            "improvement_focus": "what aspect this prompt improves",
            "reasoning": "why this improvement should help"
        }
    ]
}
```

This can work quite well on simpler datasets. It doesn't do so well on HotPot as we will see.

The MetaPromptOptimizer will try a number of rounds to try to find the best prompt.

In [21]:
from opik_optimizer import (
    MetaPromptOptimizer,
)

optimizer = MetaPromptOptimizer(
    project_name=project_name,
    model="openai/gpt-4o-mini",  # Using gpt-4o-mini for evaluation for speed
    max_rounds=1,  # Number of optimization rounds
    num_prompts_per_round=2,  # Number of prompts to generate per round
    improvement_threshold=0.01,  # Minimum improvement required to continue
    temperature=0.1,  # Lower temperature for more focused responses
    max_completion_tokens=5000,  # Maximum tokens for model completion
    num_threads=1,  # Number of threads for parallel evaluation
    subsample_size=20,  # Fixed subsample size
)


We won't do too many rounds, as this is an impossible problem without tools.

In [22]:
result2 = optimizer.optimize_prompt(
    prompt=initial_prompt,
    dataset=opik_dataset,
    metric=levenshtein_ratio,
    auto_continue=False,
    n_samples=20,  # Explicitly set
    use_subsample=True,  # Force using subsample for evaluation rounds
)

╭────────────────────────────────────────────────────────────────────╮
│ ● Running Opik Evaluation - MetaPromptOptimizer                    │
╰────────────────────────────────────────────────────────────────────╯


> Let's optimize the prompt:

╭─ system ───────────────────────────────────────────────────────────╮
│                                                                    │
│  Provide an answer to the question                                 │
│                                                                    │
╰────────────────────────────────────────────────────────────────────╯
╭─ user ─────────────────────────────────────────────────────────────╮
│                                                                    │
│  {answer}                                                          │
│                                                                    │
╰────────────────────────────────────────────────────────────────────╯

Using MetaPromptOptimizer with the parameter

KeyboardInterrupt: 

In [ ]:
result2.display()

╔═════════════════════════════════════════════ Optimization Complete ═════════════════════════════════════════════╗
║                                                                                                                 ║
║ Optimizer:         MetaPromptOptimizer                                                                          ║
║ Model Used:        openai/gpt-4o-mini (Temp: 0.1)                                                               ║
║ Metric Evaluated:  levenshtein_ratio                                                                            ║
║ Initial Score:     0.0719                                                                                       ║
║ Final Best Score:  0.4511                                                                                       ║
║ Total Improvement: 527.11%                                                                                      ║
║ Rounds Completed:  3                                                  

## MiproOptimizer

MIPRO (Multi-Iteration Prompt Optimization) is an optimizer algorithm that refines both prompts and few-shot examples in a multi-stage LLM program. It works by generating, evaluating, and refining prompts to improve language model performance. MIPRO is a more advanced method than simply "prompt hacking," offering real optimization of LLM workflows.

This sophisticated method optimizes both instructions and examples together. Using Bayesian optimization (like the FewShotBayesianOptimizer), it finds the best combinations of both elements. Through multiple testing rounds, it creates an optimized prompt that pairs effective instructions with relevant examples.

For thi first optimization, we aren't going to give it any tools to work with. Let's see how it works:

In [23]:
from opik_optimizer import MiproOptimizer

optimizer = MiproOptimizer(
    model="openai/gpt-4o-mini",  # LiteLLM or OpenAI name
    project_name=project_name,
    temperature=0.1,
    num_threads=16,
)

In order to use the MiproOptimizer we will need to define a task config

In [24]:

from opik_optimizer import TaskConfig

system_prompt = initial_prompt.get_messages()[0]["content"]

task_config = TaskConfig(
    instruction_prompt=system_prompt,
    input_dataset_fields=["question"],
    output_dataset_field="answer",
    use_chat_prompt=True,
)


In [25]:
result3 = optimizer.optimize_prompt(
    dataset=opik_dataset,
    metric=levenshtein_ratio,
    task_config=task_config,
    n_samples=10,
    num_candidates=2,
    num_trials=1,
    auto="light",
)


RUNNING WITH THE FOLLOWING LIGHT AUTO RUN SETTINGS:
num_trials: 7
minibatch: False
num_candidates: 5
valset size: 8


==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
These will be used as few-shot example candidates for our program and for creating instructions.

Bootstrapping N=5 sets of demonstrations...
Bootstrapping set 1/5
Bootstrapping set 2/5
Bootstrapping set 3/5


  0%|          | 0/2 [00:00<?, ?it/s]OPIK: Started logging traces to the "Default Project" project at https://www.comet.com/opik/api/v1/session/redirect/projects/?trace_id=0197411f-2ee9-7bc1-9f5d-4f3de986aad9&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==.
OPIK: Started logging traces to the "optimize-workshop-2025" project at https://www.comet.com/opik/api/v1/session/redirect/projects/?trace_id=0197411f-2eeb-7b3c-9c99-7cee1188f766&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==.
OPIK: Started logging traces to the "Default Project" project at https://www.comet.com/opik/api/v1/session/redirect/projects/?trace_id=0197411f-2eee-724d-a51e-abe27c58de00&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==.
100%|██████████| 2/2 [00:00<00:00, 293.85it/s]


Bootstrapped 2 full traces after 1 examples for up to 1 rounds, amounting to 2 attempts.
Bootstrapping set 4/5


  0%|          | 0/2 [00:00<?, ?it/s]OPIK: Started logging traces to the "optimize-workshop-2025" project at https://www.comet.com/opik/api/v1/session/redirect/projects/?trace_id=0197411f-2ef0-7c47-a918-dfb03efc5e26&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==.
OPIK: Started logging traces to the "Default Project" project at https://www.comet.com/opik/api/v1/session/redirect/projects/?trace_id=0197411f-2ef3-79ef-96a6-56a85fecebc3&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==.
OPIK: Started logging traces to the "optimize-workshop-2025" project at https://www.comet.com/opik/api/v1/session/redirect/projects/?trace_id=0197411f-2ef5-7122-9615-e997e11f461e&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==.
OPIK: Started logging traces to the "Default Project" project at https://www.comet.com/opik/api/v1/session/redirect/projects/?trace_id=0197411f-2ef8-7125-a86b-1e8749e2d65a&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==.
100%|██████████| 2/2 [00:00<00:00, 356.95it/s]


Bootstrapped 2 full traces after 1 examples for up to 1 rounds, amounting to 2 attempts.
Bootstrapping set 5/5


...

Average Metric: 0.88 / 4 (21.9%):  38%|███▊      | 3/8 [00:00<00:01,  3.09it/s]

OPIK: Started logging traces to the "optimize-workshop-2025" project at https://www.comet.com/opik/api/v1/session/redirect/projects/?trace_id=0197411f-4d70-7baa-a4b9-06148da3857e&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==.
OPIK: Started logging traces to the "optimize-workshop-2025" project at https://www.comet.com/opik/api/v1/session/redirect/projects/?trace_id=0197411f-4d7c-733b-b831-5846408ea8b3&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==.
OPIK: Started logging traces to the "Default Project" project at https://www.comet.com/opik/api/v1/session/redirect/projects/?trace_id=0197411f-514c-7bf5-a7c9-c24dc5b3ce46&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==.
OPIK: Started logging traces to the "Default Project" project at https://www.comet.com/opik/api/v1/session/redirect/projects/?trace_id=0197411f-514e-7687-8cfb-21dc7ffae63d&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==.


Average Metric: 1.50 / 6 (25.0%):  62%|██████▎   | 5/8 [00:00<00:00,  3.09it/s]

OPIK: Started logging traces to the "optimize-workshop-2025" project at https://www.comet.com/opik/api/v1/session/redirect/projects/?trace_id=0197411f-4d73-7635-8f95-0be29712356b&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==.
OPIK: Started logging traces to the "Default Project" project at https://www.comet.com/opik/api/v1/session/redirect/projects/?trace_id=0197411f-5159-7042-b5b9-04dcf0d0a811&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==.
OPIK: Started logging traces to the "Default Project" project at https://www.comet.com/opik/api/v1/session/redirect/projects/?trace_id=0197411f-515b-79aa-955f-3ace36124971&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==.


Average Metric: 1.78 / 8 (22.3%): 100%|██████████| 8/8 [00:00<00:00,  8.03it/s]
Score: 22.26 with parameters ['Predictor 0: Instruction 2', 'Predictor 0: Few-Shot Set 4'].
Scores so far: [0.1496330137501949, 24.41, 24.41, 14.96, 9.6, 15.25, 14.96, 22.26]
Best score so far: 24.41


Returning best identified program with score 24.41!


In [26]:
result3.display()

╔═════════════════════════════════════════════ Optimization Complete ═════════════════════════════════════════════╗
║                                                                                                                 ║
║ Optimizer:         MiproOptimizer                                                                               ║
║ Model Used:        N/A (Temp: N/A)                                                                              ║
║ Metric Evaluated:  levenshtein_ratio                                                                            ║
║ Initial Score:     N/A                                                                                          ║
║ Final Best Score:  0.2441                                                                                       ║
║ Total Improvement: N/A (no initial score)                                                                       ║
║ Rounds Completed:  0                                                  

In [31]:
result3.demonstrations

[{'augmented': True,
  'question': 'What is the name of the future head coach of the University of Pittsburgh football team whom Dick Jauron replaced as head coach of the Chicago Bears on January 24, 1999?',
  'answer': 'Dave Wannstedt'},
 {'augmented': True,
  'question': 'Who was the son of British radio and television presenter and historian best known as an analyst of election results?',
  'answer': 'David Dimbleby'}]

### Agent with Tools

Now we'll try with tools. This will allow multi-prompt optimization.

First, we need a tool. We'll use this one from DSPy:

In [32]:
# Tools:
import dspy

def search_wikipedia(query: str) -> list[str]:
    """
    This agent is used to search wikipedia. It can retrieve additional details
    about a topic.
    """
    results = dspy.ColBERTv2(url="http://20.102.90.50:2017/wiki17_abstracts")(
        query, k=3
    )
    return [x["text"] for x in results]

Let's test it out on a subject:

In [33]:
search_wikipedia("Developmental Robotics")

['Developmental robotics | Developmental robotics (DevRob), sometimes called epigenetic robotics, is a scientific field which aims at studying the developmental mechanisms, architectures and constraints that allow lifelong and open-ended learning of new skills and new knowledge in embodied machines. As in human children, learning is expected to be cumulative and of progressively increasing complexity, and to result from self-exploration of the world in combination with social interaction. The typical methodological approach consists in starting from theories of human and animal development elaborated in fields such as developmental psychology, neuroscience, developmental and evolutionary biology, and linguistics, then to formalize and implement them in robots, sometimes exploring extensions or variants of them. The experimentation of those models in robots allows researchers to confront them with reality, and as a consequence developmental robotics also provides feedback and novel hypo

And it is easy to add the tools to the config. Let's go!

In [35]:
task_config.tools = [search_wikipedia]

result4 = optimizer.optimize_prompt(
    dataset=opik_dataset,
    metric=levenshtein_ratio,
    task_config=task_config,
    n_samples=10,
    auto="light",
)


RUNNING WITH THE FOLLOWING LIGHT AUTO RUN SETTINGS:
num_trials: 7
minibatch: False
num_candidates: 3
valset size: 8


==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
These will be used as few-shot example candidates for our program and for creating instructions.

Bootstrapping N=3 sets of demonstrations...
Bootstrapping set 1/3
Bootstrapping set 2/3
Bootstrapping set 3/3


 50%|█████     | 1/2 [00:03<00:03,  3.35s/it]OPIK: Started logging traces to the "optimize-workshop-2025" project at https://www.comet.com/opik/api/v1/session/redirect/projects/?trace_id=01974124-57d9-70e7-bc5a-23386e47c31f&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==.
OPIK: Started logging traces to the "Default Project" project at https://www.comet.com/opik/api/v1/session/redirect/projects/?trace_id=01974124-7a30-7260-8171-9be57385aca9&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==.
100%|██████████| 2/2 [00:12<00:00,  6.07s/it]


Bootstrapped 2 full traces after 1 examples for up to 1 rounds, amounting to 2 attempts.

==> STEP 2: PROPOSE INSTRUCTION CANDIDATES <==
We will use the few-shot examples from the previous step, a generated dataset summary, a summary of the program code, and a randomly selected prompting tip to propose instructions.
SOURCE CODE: StringSignature(question, trajectory -> next_thought, next_tool_name, next_tool_args
    instructions="Provide an answer to the question\n\nYou are an Agent. In each episode, you will be given the fields `question` as input. And you can see your past trajectory so far.\nYour goal is to use one or more of the supplied tools to collect any necessary information for producing `answer`.\n\nTo do this, you will interleave next_thought, next_tool_name, and next_tool_args in each turn, and also when finishing the task.\nAfter each tool call, you receive a resulting observation, which gets appended to your trajectory.\n\nWhen writing next_thought, you may reason about 

OPIK: Started logging traces to the "optimize-workshop-2025" project at https://www.comet.com/opik/api/v1/session/redirect/projects/?trace_id=01974124-7a3f-7c6f-818e-fa7806318b98&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==.


DATA SUMMARY: The dataset primarily consists of questions targeting specific individuals or companies, aiming to extract concise and factual information. This structure indicates a focus on delivering clear trivia or insights related to notable figures and business operations.

Proposing instructions...

Using a randomly generated configuration for our grounded proposer.
Selected tip: persona
PROGRAM DESCRIPTION: The program appears to be designed to solve information retrieval tasks using a language model that interacts with external tools, specifically a Wikipedia search tool. It operates by taking a question as input and maintaining a trajectory of prior actions and thoughts throughout the process. The agent uses the trajectory to inform its reasoning and decide on the next steps to gather necessary information.

The program functions in a step-wise manner, where it generates a `next_thought` based on the current state and then selects a `next_tool_name` (either to search Wikipedia 

...

Average Metric: 2.59 / 6 (43.1%):  62%|██████▎   | 5/8 [00:04<00:01,  1.64it/s]

OPIK: Started logging traces to the "optimize-workshop-2025" project at https://www.comet.com/opik/api/v1/session/redirect/projects/?trace_id=01974127-2aa4-7b3f-8b72-5079013f3abb&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==.
OPIK: Started logging traces to the "Default Project" project at https://www.comet.com/opik/api/v1/session/redirect/projects/?trace_id=01974127-4016-7209-9167-d8869e196f1b&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==.


Average Metric: 2.80 / 7 (40.0%):  88%|████████▊ | 7/8 [00:05<00:00,  2.14it/s]

OPIK: Started logging traces to the "optimize-workshop-2025" project at https://www.comet.com/opik/api/v1/session/redirect/projects/?trace_id=01974127-2ab2-7cb3-a237-a922a9b069a1&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==.
OPIK: Started logging traces to the "Default Project" project at https://www.comet.com/opik/api/v1/session/redirect/projects/?trace_id=01974127-4326-753a-9779-764fe30e1c90&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==.


Average Metric: 3.80 / 8 (47.5%): 100%|██████████| 8/8 [00:06<00:00,  1.27it/s]
Score: 47.53 with parameters ['Predictor 0: Instruction 1', 'Predictor 0: Few-Shot Set 1', 'Predictor 1: Instruction 2', 'Predictor 1: Few-Shot Set 0'].
Scores so far: [0.5286088270433622, 23.92, 16.65, 57.28, 51.34, 21.59, 51.34, 47.53]
Best score so far: 57.28


Returning best identified program with score 57.28!


In [36]:
result4.display()

╔═════════════════════════════════════════════ Optimization Complete ═════════════════════════════════════════════╗
║                                                                                                                 ║
║ Optimizer:         MiproOptimizer                                                                               ║
║ Model Used:        N/A (Temp: N/A)                                                                              ║
║ Metric Evaluated:  levenshtein_ratio                                                                            ║
║ Initial Score:     N/A                                                                                          ║
║ Final Best Score:  0.5728                                                                                       ║
║ Total Improvement: N/A (no initial score)                                                                       ║
║ Rounds Completed:  0                                                  

In [37]:
result4.demonstrations

[{'id': '0197044c-d734-70e8-8ce6-69acdd03e2d4',
  'question': "Rectrix Aviation's jet charter business involves what service?",
  'answer': 'renting an entire aircraft',
  'dspy_uuid': 'ee037613-de5a-44a1-acf2-9dbcfa7a76e2',
  'dspy_split': 'train'},
 {'id': '0197044c-d71d-7be5-8188-c334019ba38e',
  'question': 'whos family had their own reality tv show. Robert Kardashian or Manvel Gamburyan?',
  'answer': 'their family reality television series',
  'dspy_uuid': 'ed124b43-ddcd-4ea3-a3af-6a9a2b722e6a',
  'dspy_split': 'train'}]

## Using Optimized Prompts

Recall:

1. result1 - FewShotBayesianOptimizer
2. result2 - MetaPromptOptimizer
3. result3 - MiproOptimizer (no tools)
4. result4 - MiproOptimizer (with search_wikipedia)

How can we use the optimized results?

For the first one, recall that the fewshot examples are here:

In [38]:
result1.details["chat_messages"]

[{'role': 'system',
  'content': 'Provide an answer to the question\n\n### Few Shot Examples\nQuestion: What novel did an author who was featured in the Voices of Ghana and later became a journalist write?\nAnswer: The Gab Boys\n\nQuestion: What were the dates of the campaign that eventually led to the Treaty of Roskilde?\nAnswer: between 30 January and 8 February 1658\n\nQuestion: Are Wanding Town and Jiujiang, Guangdong both frontier towns?\nAnswer: no\n\nQuestion: The Knicks–Nuggets was the most penalized on-court fight in the NBA since an altercation that occured in what city?\nAnswer: Auburn Hills'},
 {'role': 'user', 'content': '{answer}'}]

So, once we have those we can do the following:

In [39]:
from litellm.integrations.opik.opik import OpikLogger
import litellm
opik_logger = OpikLogger()
litellm.callbacks = [opik_logger]

def query(question, chat_messages):
    messages = chat_messages[:-1] # Cut off the last one
    # replace it with question in proper format:
    messages.append({'role': 'user', 'content': '{"question": "%s"}"}' % question})

    response = litellm.completion(
        model="gpt-4o-mini",
        temperature=0.1,
        max_tokens=5000,
        messages=messages,
    )
    return response.choices[0].message.content

In [40]:
query("When was David Chalmers born?", result1.details["chat_messages"])

'Answer: April 20, 1966'

In [41]:
query("What weighs more: a baby elephant or an SUV?", result1.details["chat_messages"])

'Answer: A baby elephant typically weighs more than an SUV.'

If it says "elephant" that is not correct!

Let's try that same question with a tool:

In [42]:
result = result4.details["program"](question="What weighs more: a baby elephant or an SUV?")
result.answer

OPIK: Started logging traces to the "optimize-workshop-2025" project at https://www.comet.com/opik/api/v1/session/redirect/projects/?trace_id=019745a0-8fa4-7754-892b-14c7616f85f2&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==.


'An SUV weighs more than a baby elephant.'

Well done optimizer!

We'll now head back to the slides to summarize the workshop.

# Resources

1. [Opik Optimizer Workshop Slides](https://bit.ly/opik-optimizer-dsblank-slides)